# Quantitative Trading Platform - Example Usage

This notebook demonstrates how to use the quantitative trading platform with various strategies, risk management, and portfolio optimization.

## 1. Setup and Imports

In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Import platform components
from src.strategies import MovingAverageCrossoverStrategy, BollingerBandReversionStrategy
from src.indicators.technical_indicators import MovingAverage, RSI, MACD, BollingerBands
from src.portfolio.portfolio_manager import PortfolioManager
from src.risk_management.risk_manager import PortfolioRiskManager, RiskMetrics
from src.core.backtester import BacktestEngine
from src.config.config import ConfigManager
from src.data.loaders.data_loader import DataLoader

# Configure display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Platform components imported successfully!")

## 2. Load Configuration and Sample Data

In [ ]:
# Load configuration
config_manager = ConfigManager()
backtest_config = config_manager.get_backtest_config()
risk_config = config_manager.get_risk_config()

print("Configuration loaded:")
print(f"Initial Capital: ₹{backtest_config.initial_capital:,.0f}")
print(f"Commission Rate: {backtest_config.commission_rate:.3f}")
print(f"Max Portfolio Risk: {risk_config.max_portfolio_risk_percent}%")
print(f"Max Position Size: {risk_config.max_single_position_percent}%")

In [ ]:
# Generate sample data for demonstration
# In practice, you would load real market data

def generate_sample_data(symbol="RELIANCE", days=252, start_price=2500):
    """Generate sample OHLCV data with realistic price movements."""
    np.random.seed(42)
    
    dates = pd.date_range(start='2023-01-01', periods=days, freq='D')
    
    # Generate price series with trend and volatility
    returns = np.random.normal(0.0005, 0.02, days)  # Small positive drift with 2% daily volatility
    prices = [start_price]
    
    for i in range(1, days):
        price = prices[-1] * (1 + returns[i])
        prices.append(max(price, start_price * 0.5))  # Floor at 50% of start price
    
    # Generate OHLC from close prices
    data = []
    for i, (date, close) in enumerate(zip(dates, prices)):
        # Generate realistic OHLC
        daily_range = close * np.random.uniform(0.005, 0.03)  # 0.5% to 3% daily range
        
        high = close + np.random.uniform(0, daily_range)
        low = close - np.random.uniform(0, daily_range)
        open_price = (high + low) / 2 + np.random.uniform(-daily_range/4, daily_range/4)
        
        volume = np.random.randint(1000000, 5000000)  # Random volume
        
        data.append({
            'date': date,
            'symbol': symbol,
            'open': open_price,
            'high': high,
            'low': low,
            'close': close,
            'volume': volume
        })
    
    return pd.DataFrame(data)

# Generate sample data for multiple symbols
symbols = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'ICICIBANK']
sample_data = {}

for symbol in symbols:
    start_prices = {'RELIANCE': 2500, 'TCS': 3200, 'INFY': 1400, 'HDFCBANK': 1600, 'ICICIBANK': 900}
    sample_data[symbol] = generate_sample_data(symbol, days=252, start_price=start_prices[symbol])

print(f"Generated sample data for {len(symbols)} symbols:")
for symbol in symbols:
    data = sample_data[symbol]
    print(f"{symbol}: {len(data)} days, Price range: ₹{data['close'].min():.0f} - ₹{data['close'].max():.0f}")

## 3. Technical Indicators Analysis

In [ ]:
# Demonstrate technical indicators on RELIANCE data
reliance_data = sample_data['RELIANCE'].copy()

# Calculate various technical indicators
sma_20 = MovingAverage(period=20)
sma_50 = MovingAverage(period=50)
rsi = RSI(period=14)
macd = MACD(fast_period=12, slow_period=26, signal_period=9)
bb = BollingerBands(period=20, std_dev=2.0)

# Apply indicators to data
reliance_data['sma_20'] = sma_20.calculate(reliance_data['close'])
reliance_data['sma_50'] = sma_50.calculate(reliance_data['close'])
reliance_data['rsi'] = rsi.calculate(reliance_data['close'])

# MACD calculation
macd_data = macd.calculate(reliance_data['close'])
reliance_data['macd'] = macd_data['macd']
reliance_data['macd_signal'] = macd_data['signal']
reliance_data['macd_histogram'] = macd_data['histogram']

# Bollinger Bands
bb_data = bb.calculate(reliance_data['close'])
reliance_data['bb_upper'] = bb_data['upper_band']
reliance_data['bb_middle'] = bb_data['middle_band']
reliance_data['bb_lower'] = bb_data['lower_band']

print("Technical indicators calculated for RELIANCE")
print(reliance_data[['date', 'close', 'sma_20', 'sma_50', 'rsi']].tail())

In [ ]:
# Plot technical analysis chart
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Price and moving averages
axes[0].plot(reliance_data['date'], reliance_data['close'], label='Close Price', linewidth=2)
axes[0].plot(reliance_data['date'], reliance_data['sma_20'], label='SMA 20', alpha=0.7)
axes[0].plot(reliance_data['date'], reliance_data['sma_50'], label='SMA 50', alpha=0.7)
axes[0].fill_between(reliance_data['date'], reliance_data['bb_upper'], reliance_data['bb_lower'], alpha=0.2, label='Bollinger Bands')
axes[0].set_title('RELIANCE - Price and Moving Averages')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(reliance_data['date'], reliance_data['rsi'], label='RSI', color='orange')
axes[1].axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought (70)')
axes[1].axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold (30)')
axes[1].set_title('RSI (Relative Strength Index)')
axes[1].set_ylim(0, 100)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# MACD
axes[2].plot(reliance_data['date'], reliance_data['macd'], label='MACD', color='blue')
axes[2].plot(reliance_data['date'], reliance_data['macd_signal'], label='Signal', color='red')
axes[2].bar(reliance_data['date'], reliance_data['macd_histogram'], label='Histogram', alpha=0.3, color='gray')
axes[2].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[2].set_title('MACD (Moving Average Convergence Divergence)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Technical analysis charts plotted successfully!")

## 4. Strategy Backtesting

In [ ]:
# Initialize and backtest Moving Average Crossover Strategy
ma_strategy = MovingAverageCrossoverStrategy(
    fast_period=10,
    slow_period=20,
    position_size=50000,  # ₹50,000 per trade
    stop_loss_pct=0.02,   # 2% stop loss
    take_profit_pct=0.04  # 4% take profit
)

# Initialize backtester
backtester = BacktestEngine(
    initial_capital=backtest_config.initial_capital,
    commission_rate=backtest_config.commission_rate,
    slippage_rate=backtest_config.slippage_rate
)

print("Running Moving Average Crossover Strategy backtest...")

# Run backtest
backtest_results = backtester.run_backtest(
    strategy=ma_strategy,
    data=reliance_data,
    start_date=reliance_data['date'].iloc[0],
    end_date=reliance_data['date'].iloc[-1]
)

print("\nBacktest Results:")
print(f"Total Return: {backtest_results['total_return']:.2%}")
print(f"Annualized Return: {backtest_results['annualized_return']:.2%}")
print(f"Sharpe Ratio: {backtest_results['sharpe_ratio']:.2f}")
print(f"Maximum Drawdown: {backtest_results['max_drawdown']:.2%}")
print(f"Number of Trades: {backtest_results['num_trades']}")
print(f"Win Rate: {backtest_results['win_rate']:.2%}")

In [ ]:
# Plot backtest results
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Portfolio value over time
portfolio_history = backtest_results['portfolio_history']
dates = [record['date'] for record in portfolio_history]
values = [record['total_value'] for record in portfolio_history]

axes[0].plot(dates, values, label='Portfolio Value', linewidth=2)
axes[0].axhline(y=backtest_config.initial_capital, color='r', linestyle='--', alpha=0.7, label='Initial Capital')
axes[0].set_title('Portfolio Value Over Time - MA Crossover Strategy')
axes[0].set_ylabel('Portfolio Value (₹)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Drawdown
initial_value = values[0]
running_max = np.maximum.accumulate(values)
drawdown = [(val - running_max[i]) / running_max[i] * 100 for i, val in enumerate(values)]

axes[1].fill_between(dates, drawdown, 0, alpha=0.3, color='red', label='Drawdown')
axes[1].plot(dates, drawdown, color='red', linewidth=1)
axes[1].set_title('Portfolio Drawdown')
axes[1].set_ylabel('Drawdown (%)')
axes[1].set_xlabel('Date')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Risk Management Analysis

In [ ]:
# Initialize risk manager
risk_manager = PortfolioRiskManager(config=risk_config)

# Calculate returns from portfolio history
portfolio_values = [record['total_value'] for record in portfolio_history]
daily_returns = []

for i in range(1, len(portfolio_values)):
    daily_return = (portfolio_values[i] - portfolio_values[i-1]) / portfolio_values[i-1]
    daily_returns.append(daily_return)

# Update risk manager with portfolio data
risk_manager.daily_returns = daily_returns
risk_manager.portfolio_value = portfolio_values[-1]

# Calculate risk metrics
returns_series = pd.Series(daily_returns)
risk_metrics = {
    'VaR_95': RiskMetrics.value_at_risk(returns_series, 0.95),
    'VaR_99': RiskMetrics.value_at_risk(returns_series, 0.99),
    'CVaR_95': RiskMetrics.conditional_var(returns_series, 0.95),
    'Sharpe_Ratio': RiskMetrics.sharpe_ratio(returns_series),
    'Sortino_Ratio': RiskMetrics.sortino_ratio(returns_series),
    'Max_Drawdown': RiskMetrics.maximum_drawdown(pd.Series(portfolio_values))['max_drawdown']
}

print("Risk Analysis Results:")
print("-" * 40)
for metric, value in risk_metrics.items():
    if 'Ratio' in metric:
        print(f"{metric}: {value:.2f}")
    else:
        print(f"{metric}: {value:.2%}")

In [ ]:
# Plot risk analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Returns distribution
axes[0, 0].hist(daily_returns, bins=30, alpha=0.7, density=True)
axes[0, 0].axvline(risk_metrics['VaR_95'], color='r', linestyle='--', label=f'VaR 95%: {risk_metrics["VaR_95"]:.2%}')
axes[0, 0].axvline(risk_metrics['VaR_99'], color='orange', linestyle='--', label=f'VaR 99%: {risk_metrics["VaR_99"]:.2%}')
axes[0, 0].set_title('Daily Returns Distribution')
axes[0, 0].set_xlabel('Daily Return')
axes[0, 0].set_ylabel('Density')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Rolling volatility
rolling_vol = pd.Series(daily_returns).rolling(window=20).std() * np.sqrt(252) * 100
axes[0, 1].plot(rolling_vol.index, rolling_vol.values)
axes[0, 1].set_title('Rolling 20-Day Volatility (Annualized)')
axes[0, 1].set_xlabel('Days')
axes[0, 1].set_ylabel('Volatility (%)')
axes[0, 1].grid(True, alpha=0.3)

# Cumulative returns vs drawdown
cumulative_returns = np.cumprod(1 + np.array(daily_returns)) - 1
axes[1, 0].plot(cumulative_returns * 100, label='Cumulative Return')
axes[1, 0].fill_between(range(len(drawdown[1:])), np.array(drawdown[1:]), 0, alpha=0.3, color='red', label='Drawdown')
axes[1, 0].set_title('Cumulative Returns vs Drawdown')
axes[1, 0].set_xlabel('Days')
axes[1, 0].set_ylabel('Return/Drawdown (%)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Risk metrics comparison
metrics_names = ['Sharpe Ratio', 'Sortino Ratio']
metrics_values = [risk_metrics['Sharpe_Ratio'], risk_metrics['Sortino_Ratio']]
axes[1, 1].bar(metrics_names, metrics_values, color=['blue', 'green'], alpha=0.7)
axes[1, 1].set_title('Risk-Adjusted Return Metrics')
axes[1, 1].set_ylabel('Ratio')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Portfolio Management Example

In [ ]:
# Initialize portfolio manager
portfolio_manager = PortfolioManager(initial_cash=backtest_config.initial_capital)

# Simulate some trades
from src.common.base_classes import Order, OrderSide, OrderType

# Create sample orders
sample_orders = [
    Order('RELIANCE', OrderSide.BUY, OrderType.MARKET, 20),
    Order('TCS', OrderSide.BUY, OrderType.MARKET, 15),
    Order('INFY', OrderSide.BUY, OrderType.MARKET, 35),
    Order('HDFCBANK', OrderSide.BUY, OrderType.MARKET, 30),
    Order('ICICIBANK', OrderSide.BUY, OrderType.MARKET, 55)
]

# Execute orders with current prices
current_prices = {symbol: data['close'].iloc[-1] for symbol, data in sample_data.items()}
portfolio_manager.update_prices(current_prices)

for order in sample_orders:
    fill_price = current_prices[order.symbol]
    commission = fill_price * order.quantity * 0.001  # 0.1% commission
    
    success = portfolio_manager.execute_order(order, fill_price, commission)
    if success:
        print(f"Executed: {order.side.value} {order.quantity} shares of {order.symbol} at ₹{fill_price:.2f}")

# Get portfolio summary
portfolio_summary = portfolio_manager.get_portfolio_summary()

print("\nPortfolio Summary:")
print("-" * 50)
print(f"Total Portfolio Value: ₹{portfolio_summary['portfolio_value']:,.2f}")
print(f"Cash: ₹{portfolio_summary['cash']:,.2f}")
print(f"Number of Positions: {portfolio_summary['num_positions']}")
print(f"Number of Trades: {portfolio_summary['num_trades']}")

print("\nPositions:")
for position in portfolio_summary['positions']:
    symbol = position['symbol']
    quantity = position['quantity']
    avg_price = position['avg_price']
    market_value = position['market_value']
    unrealized_pnl = position['unrealized_pnl']
    
    print(f"{symbol}: {quantity} shares @ ₹{avg_price:.2f} | Market Value: ₹{market_value:,.2f} | P&L: ₹{unrealized_pnl:,.2f}")

## 7. Strategy Comparison

In [ ]:
# Compare different strategies
strategies = {
    'MA_Crossover_10_20': MovingAverageCrossoverStrategy(fast_period=10, slow_period=20, position_size=50000),
    'MA_Crossover_5_15': MovingAverageCrossoverStrategy(fast_period=5, slow_period=15, position_size=50000),
    'BB_Reversion': BollingerBandReversionStrategy(bb_period=20, position_size=50000)
}

strategy_results = {}

print("Comparing strategies on RELIANCE data...")
print("-" * 60)

for strategy_name, strategy in strategies.items():
    # Create new backtester for each strategy
    backtester = BacktestEngine(
        initial_capital=backtest_config.initial_capital,
        commission_rate=backtest_config.commission_rate,
        slippage_rate=backtest_config.slippage_rate
    )
    
    # Run backtest
    results = backtester.run_backtest(
        strategy=strategy,
        data=reliance_data,
        start_date=reliance_data['date'].iloc[0],
        end_date=reliance_data['date'].iloc[-1]
    )
    
    strategy_results[strategy_name] = results
    
    print(f"\n{strategy_name}:")
    print(f"  Total Return: {results['total_return']:.2%}")
    print(f"  Sharpe Ratio: {results['sharpe_ratio']:.2f}")
    print(f"  Max Drawdown: {results['max_drawdown']:.2%}")
    print(f"  Win Rate: {results['win_rate']:.2%}")
    print(f"  Num Trades: {results['num_trades']}")

In [ ]:
# Create comparison table
comparison_data = []
for strategy_name, results in strategy_results.items():
    comparison_data.append({
        'Strategy': strategy_name,
        'Total Return (%)': results['total_return'] * 100,
        'Annualized Return (%)': results['annualized_return'] * 100,
        'Sharpe Ratio': results['sharpe_ratio'],
        'Max Drawdown (%)': results['max_drawdown'] * 100,
        'Win Rate (%)': results['win_rate'] * 100,
        'Num Trades': results['num_trades'],
        'Final Value': results['final_portfolio_value']
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nStrategy Comparison Table:")
print(comparison_df.round(2))

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Total returns comparison
axes[0, 0].bar(comparison_df['Strategy'], comparison_df['Total Return (%)'], alpha=0.7)
axes[0, 0].set_title('Total Return Comparison')
axes[0, 0].set_ylabel('Return (%)')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# Sharpe ratio comparison
axes[0, 1].bar(comparison_df['Strategy'], comparison_df['Sharpe Ratio'], alpha=0.7, color='green')
axes[0, 1].set_title('Sharpe Ratio Comparison')
axes[0, 1].set_ylabel('Sharpe Ratio')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# Max drawdown comparison
axes[1, 0].bar(comparison_df['Strategy'], comparison_df['Max Drawdown (%)'], alpha=0.7, color='red')
axes[1, 0].set_title('Maximum Drawdown Comparison')
axes[1, 0].set_ylabel('Max Drawdown (%)')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# Win rate comparison
axes[1, 1].bar(comparison_df['Strategy'], comparison_df['Win Rate (%)'], alpha=0.7, color='orange')
axes[1, 1].set_title('Win Rate Comparison')
axes[1, 1].set_ylabel('Win Rate (%)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nStrategy comparison completed successfully!")

## 8. Summary and Next Steps

In [ ]:
print("🎉 Quantitative Trading Platform Demo Completed!")
print("=" * 60)
print("\n📊 What we demonstrated:")
print("• Technical indicators calculation and visualization")
print("• Strategy implementation and backtesting")
print("• Risk management and portfolio analysis")
print("• Strategy comparison and performance metrics")
print("• Portfolio management with multiple positions")

print("\n🚀 Next steps to enhance the platform:")
print("• Integrate real market data feeds (NSE/BSE)")
print("• Implement live trading with broker APIs")
print("• Add machine learning models for prediction")
print("• Create more sophisticated strategies")
print("• Build web dashboard for monitoring")
print("• Add options and futures trading capabilities")
print("• Implement paper trading for strategy testing")

print("\n💡 Pro Tips:")
print("• Always backtest strategies on out-of-sample data")
print("• Consider transaction costs and slippage")
print("• Implement proper risk management")
print("• Monitor strategy performance regularly")
print("• Diversify across multiple strategies and assets")

print("\n🎯 Happy Trading! 📈")